# Section 0 Defining modules/libraries/functions

In [1]:
import glob
import pandas as pd
import numpy as np
from pathlib import Path
pd.options.display.max_rows = 4000


In [2]:
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
            " columns that have missing values.")
    return mis_val_table_ren_columns


# Section 1 Importing Data

In [3]:
cropsCodes = pd.read_csv("C:/Users/cianw/Documents/dataAnalytics/CA2/Data/Eurostat/Code Dictionary/crops.dic", sep='\t',names=['crops', 'crop_name'], header = None)
strucproCodes = pd.read_csv("C:/Users/cianw/Documents/dataAnalytics/CA2/Data/Eurostat/Code Dictionary/strucpro.dic",sep='\t',names=['code', 'units'], header = None)
unitCodes = pd.read_csv("C:/Users/cianw/Documents/dataAnalytics/CA2/Data/Eurostat/Code Dictionary/unit.dic",sep='\t',names=['code', 'units'], header = None)

In [4]:

#Organic Production Data
stdProduction_path = Path(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Eurostat\Agricultural Production\Crops\apro_cpsh1_linear.csv")
orgTonne_path= Path(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Eurostat\Agricultural Production\Crops\org_croppro_linear.csv")
orgArea_path = Path(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Eurostat\Agricultural Production\Crops\org_cropar_linear.csv")

#Harmonised Risk Index 1 Data
hriPath = Path(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Eurostat\Agricultural Production\Crops\Pesiticide Use Risk Indicator\aei_hri_linear.csv")

#Organic Processors Data
orgProcessors_path = Path(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Eurostat\Agricultural Production\Crops\Organic Processors\org_cpreact_linear.csv")

#Organic Area Utilisation Data
orgAreaUtil_path = Path(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Eurostat\Agricultural Production\Crops\Organic Area\sdg_02_40_linear.csv")

#N and P Fertilizer Data
fertUse_path = Path(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Eurostat\Agricultural Production\Crops\Fertlizer Use\aei_fm_usefert_linear.csv")

#Waste Generation Data
wasteGeneration_path = Path(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Eurostat\Resource Usage\cei_pc034_linear.csv")

#National Productivity Data
productivityIndex_Path = Path(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Eurostat\Resource Usage\cei_pc030_linear.csv")

#Country Gini Data
countryGini_path = Path(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Eurostat\Quality of Life\tessi190_linear.csv")

#Employment Rate Data
employmentRate_Path = Path(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Eurostat\Economics\tesem010_linear.csv")

#Median/Mean Income Data
income_Path = Path(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Eurostat\Economics\ilc_di03_linear.csv")

#Biodiversity of Birds Data
birdBiodiversity_Path = Path(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Eurostat\Biodiversity Index\env_bio2_linear.csv")

#Pesticide Sales
pestSales_Path = Path(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Eurostat\Agricultural Production\Crops\Pesticide Sales\aei_fm_salpest09_linear.csv")

#Pesticide Use
pestUse_Path = Path(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Eurostat\Agricultural Production\Crops\aei_pestuse_linear.csv")

#Farm Structure
farmStructure_Path = Path(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Eurostat\Farm Structure\ef_lac_main_linear.csv");

In [5]:
#Harmonised Risk Index 1 Data
hriPesticide = pd.read_csv(hriPath)
hriPesticide = hriPesticide[hriPesticide['subst_cat'].isin(['HRI1'])]
hriPesticide = hriPesticide[~hriPesticide['geo'].isin(['EU', 'EU27_2020', 'EU28'])]
hriPesticide = hriPesticide.drop(columns = ['DATAFLOW', 'LAST UPDATE', 'freq', 'unit', 'OBS_FLAG'])
hriPesticide = hriPesticide.rename(columns={"OBS_VALUE":"harmRiskInd"})
hriPesticide['harmRiskInd_next'] = hriPesticide.groupby(['geo'])['harmRiskInd'].shift(-1)
hriPesticide['increase'] = np.where((hriPesticide['harmRiskInd'] <= hriPesticide['harmRiskInd_next']), 1, 0)
hriPesticide['forAnalysis'] = hriPesticide['harmRiskInd_next'].apply(lambda x: 0 if pd.isnull(x) else 1)
#print(hriPesticide.groupby('TIME_PERIOD').geo.nunique())
#hriPesticide.shape
#hriPesticide.info()
hriPesticide.head(25)

,subst_cat,geo,TIME_PERIOD,harmRiskInd,harmRiskInd_next,increase,forAnalysis
638,HRI1,AT,2011,102,106.0,1,1
639,HRI1,AT,2012,106,92.0,0,1
640,HRI1,AT,2013,92,95.0,1,1
641,HRI1,AT,2014,95,101.0,1,1
642,HRI1,AT,2015,101,115.0,1,1
643,HRI1,AT,2016,115,116.0,1,1
644,HRI1,AT,2017,116,129.0,1,1
645,HRI1,AT,2018,129,121.0,0,1
646,HRI1,AT,2019,121,135.0,1,1
647,HRI1,AT,2020,135,NaN,0,0


In [6]:
#Organic Area Utilisation Data
orgAreaUtil = pd.read_csv(orgAreaUtil_path)
orgAreaUtil = orgAreaUtil.drop(columns = ['DATAFLOW', 'LAST UPDATE', 'crops','freq', 'agprdmet', 'unit', 'OBS_FLAG'])
orgAreaUtil = orgAreaUtil.rename(columns={"OBS_VALUE":"areaUsedForOrganic_PCT"})
#orgAreaUtil.shape
#orgAreaUtil.info()
#orgAreaUtil.head(10)

Can apply an interesting step here by using left joins to exclude unwanted data later.

The dataset imported and pivoted/transposed below contains more than just the Countries required, it contains subdivisions indicated by numbers beside the country name. Instead of manually writing code that excludes these cases, this set will be left joined later, which will exclude the data automatically.

In [7]:
#N and P Fertilizer Data
fertUse = pd.read_csv(fertUse_path)
fertUse = fertUse.drop(columns = ['DATAFLOW', 'LAST UPDATE', 'freq', 'unit', 'OBS_FLAG'])
fertUse = fertUse.pivot(index=['geo', 'TIME_PERIOD'], columns='nutrient', values='OBS_VALUE').reset_index() 
fertUse = fertUse.rename(columns={"N":"N_use_tonne",
                                 "P":"P_use_tonne"})
#print(fertUseGroup3.groupby('TIME_PERIOD').geo.nunique())
#fertUse.shape
#fertUse.info()

Below I am imputing the missing odd years by averaging the two around it. By treating the numbers as a trend.

In [8]:
#Waste Generation Data
wasteGeneration = pd.read_csv(wasteGeneration_path)
wasteGeneration = wasteGeneration.drop(columns = ['DATAFLOW', 'LAST UPDATE','nace_r2', 'unit','freq', 'OBS_FLAG', 'hazard', 'waste'])
wasteGeneration["waste_unit"] = "KG per Capita"
wasteGeneration = wasteGeneration.rename(columns={"OBS_VALUE":"waste"})

wasteGeneration_lag = wasteGeneration
wasteGeneration_lag['TIME_LAG'] = wasteGeneration_lag.groupby(['geo'])['TIME_PERIOD'].shift(1)
wasteGeneration_lag['waste_lag'] = wasteGeneration_lag.groupby(['geo'])['waste'].shift(1)

wasteGeneration_lag['waste_temp'] = ((wasteGeneration_lag['waste'] + wasteGeneration_lag['waste_lag'])/2)
wasteGeneration_lag['TIME_PERIOD_temp'] = ((wasteGeneration_lag['TIME_PERIOD'] + wasteGeneration_lag['TIME_LAG'])/2)
wasteGeneration_lag = wasteGeneration_lag.drop(columns = ['TIME_PERIOD', 'TIME_LAG','waste', 'waste_lag'])
wasteGeneration_lag = wasteGeneration_lag.dropna()
wasteGeneration_lag = wasteGeneration_lag.rename(columns={"waste_temp":"waste",
                                                           "TIME_PERIOD_temp":"TIME_PERIOD"})
wasteGeneration_lag['TIME_PERIOD'] = wasteGeneration_lag['TIME_PERIOD'].astype('int')
wasteGeneration = pd.concat([wasteGeneration, wasteGeneration_lag], join='inner', ignore_index=True)

#wasteGeneration.shape
#wasteGeneration.info()

In [9]:
#National Productivity Data
productivityIndex = pd.read_csv(productivityIndex_Path)
productivityIndex  = productivityIndex[productivityIndex['unit'].isin(['PPS_KG'])]
productivityIndex = productivityIndex.drop(columns = ['DATAFLOW', 'LAST UPDATE', 'unit','freq', 'OBS_FLAG'])
productivityIndex["productivity_unit"] = "Purchase Power Standard Per KG"
productivityIndex = productivityIndex.rename(columns={"OBS_VALUE":"productivity"})
#productivityIndex.shape
#productivityIndex.info()
#productivityIndex.head(20)

In [10]:
#GINI index data
countryGini = pd.read_csv(countryGini_path)

countryGini = countryGini.drop(columns = ['DATAFLOW', 'LAST UPDATE', 'freq', 'indic_il', 'OBS_FLAG'])
countryGini = countryGini.rename(columns={"OBS_VALUE":"gini"})

#countryGini.shape
#countryGini.info()

In [11]:
#Employment Rate Data
emplyomentRate = pd.read_csv(employmentRate_Path)
emplyomentRate = emplyomentRate.drop(columns = ['DATAFLOW', 'LAST UPDATE', 'freq', 'OBS_FLAG', 'age', 'unit', 'indic_em'])
emplyomentRate = emplyomentRate.pivot(index=['geo', 'TIME_PERIOD'], columns='sex', values='OBS_VALUE').reset_index() 
emplyomentRate = emplyomentRate.rename(columns={"T":"emplyomentRate_T",
                                               "M":"emplyomentRate_M",
                                               "F":"emplyomentRate_F"})
#emplyomentRate.shape
#emplyomentRate.info()

In [12]:
#Median/Mean Income Data
income = pd.read_csv(income_Path)
income  = income[income['unit'].isin(['EUR'])]
income  = income[income['age'].isin(['TOTAL'])]
income = income.drop(columns = ['DATAFLOW', 'LAST UPDATE', 'freq', 'OBS_FLAG', 'age', 'unit'])
income = income.pivot(index=['geo', 'TIME_PERIOD'], columns=['indic_il', 'sex'], values='OBS_VALUE').reset_index() 
income.columns = [''.join(col) for col in income.columns.values]
#income.head()
#income.shape
#income.info()


In [13]:
#Bird Biodiversity
birdBiodiversity = pd.read_csv(birdBiodiversity_Path)
birdBiodiversity = birdBiodiversity.drop(columns = ['DATAFLOW', 'LAST UPDATE', 'freq', 'OBS_FLAG', 'unit'])
birdBiodiversity = birdBiodiversity.rename(columns={"OBS_VALUE":"birdBiodiversityIndex"})

#birdBiodiversity.head()

In [14]:
# Pest Use
pestUse= pd.read_csv(pestUse_Path)
#pestUse = pestUse.drop(columns = ['DATAFLOW', 'LAST UPDATE', 'freq', 'OBS_FLAG'])
#pestSales = pestSales[(pestSales['TIME_PERIOD'].isin([2016,2018]))]

pestUse['TIME_PERIOD'] = pestUse['TIME_PERIOD'].apply(lambda x: str(x)+ '_y')
pestUse = pestUse[(pestUse['unit'].isin(['KG']))]
pestUse = pestUse[(pestUse['pesticid'].str.contains("_"))]
pestUse  = pestUse[~pestUse['OBS_FLAG'].isin(['c'])]
pestUse['OBS_VALUE'] = np.where(pestUse.OBS_FLAG == 'n', 0, pestUse.OBS_VALUE)
#pestSales = pestSales[(pestSales['pesticid'].str.isdigit()==True)]

#print(pestSales.pesticid.unique())
#pestSales.head()

pestUse = pestUse.pivot(index=['pesticid','crops'], columns =['geo', 'TIME_PERIOD'], values='OBS_VALUE').reset_index() 
pestUse.columns = ['_'.join(col) for col in pestUse.columns.values]

#pestUse.head(30)
#pestUse.shape
#missing_values_table(pestUse)


In [15]:
# Pest Sales for Stats Analysis
pestSales= pd.read_csv(pestSales_Path)
pestSales['mainFert'] = pestSales['pesticid'].str.contains(r'[0-9]')
pestSales = pestSales.dropna(subset=['OBS_VALUE'])
pestSales = pestSales.drop(pestSales[pestSales['mainFert'] == True].index)
pestSales = pestSales.drop(pestSales[pestSales['pesticid'] == 'TOTAL'].index)
pestSales = pestSales.drop(columns = ['DATAFLOW', 'LAST UPDATE', 'freq', 'OBS_FLAG', 'unit', 'mainFert'])

pestSales = pestSales.sort_values('OBS_VALUE').drop_duplicates(['geo', 'TIME_PERIOD'], keep='last')
pestSales = pestSales.rename(columns={"OBS_VALUE":"pest_KG",
                                     "pesticid" : "mostFrequentPest"})
pestSales.shape
#pestSales.head(300)


#pestSales = pestSales.pivot(index=['pesticid'], columns =['TIME_PERIOD', 'geo'], values='OBS_VALUE').reset_index() 
#pestSales.columns = ['_'.join(col) for col in pestSales.columns.values]

#pestSales.head(30)


(307, 4)

In [16]:
orgTonne = pd.read_csv(orgTonne_path)
orgTonne = orgTonne[orgTonne['crops'].str.contains("0000")]
orgTonne = orgTonne.drop(orgTonne[orgTonne['OBS_VALUE'] == 0].index)
orgTonne = orgTonne.rename(columns={"OBS_VALUE":"orgTonnes",
                                   "crops":"mostGrownOrganic"})
#orgTonne.info()
orgTonne = orgTonne[~orgTonne['OBS_FLAG'].isin(['c','n'])]
orgTonne = orgTonne.drop(columns = ['DATAFLOW', 'LAST UPDATE', 'freq', 'unit', 'OBS_FLAG'])
orgTonne = orgTonne.sort_values('orgTonnes').drop_duplicates(['geo', 'TIME_PERIOD'], keep='last')

#print(orgTonne.head(10))
#missing_values_table(orgTonne)

In [17]:
stdProduction_lin = pd.read_csv(stdProduction_path)
stdProduction_lin = stdProduction_lin[stdProduction_lin['crops'].str.contains("0000")]
stdProduction_lin = stdProduction_lin[stdProduction_lin['strucpro'].isin(['PR_HU_EU']) ]
stdProduction_lin  = stdProduction_lin[~stdProduction_lin['OBS_FLAG'].isin(['c','n'])]
stdProduction_lin = stdProduction_lin.drop(stdProduction_lin[stdProduction_lin['OBS_VALUE'] == 0].index)
stdProduction_lin['OBS_VALUE'] = stdProduction_lin['OBS_VALUE']*1000
stdProduction_lin = stdProduction_lin.rename(columns={"OBS_VALUE":"stdTonnes",
                                                      "crops":"mostGrownStd"})
#orgTonne.info()
stdProduction_lin = stdProduction_lin[~stdProduction_lin['OBS_FLAG'].isin(['c','n'])]
stdProduction_lin = stdProduction_lin.drop(columns = ['DATAFLOW', 'LAST UPDATE', 'freq', 'OBS_FLAG', 'strucpro'])
stdProduction_lin = stdProduction_lin.sort_values('stdTonnes').drop_duplicates(['geo', 'TIME_PERIOD'], keep='last')

#print(stdProduction_lin.head(10))
#missing_values_table(orgTonne)

In [18]:
stdProduction_lin = pd.read_csv(stdProduction_path)
stdProduction_lin = stdProduction_lin[stdProduction_lin['crops'].str.contains("0000")]
stdProduction_lin = stdProduction_lin[stdProduction_lin['strucpro'].isin(['AR', 'PR_HU_EU']) ]
stdProduction_lin  = stdProduction_lin[~stdProduction_lin['OBS_FLAG'].isin(['c','n'])]

In [19]:
#print(orgProduction.geo.unique())
#print(stdProduction.geo.unique())
#print(orgArea_all.geo.unique())
#print(orgTonne.geo.unique())

# Variable Creation

In [20]:
#%whos DataFrame
#hriPesticide.head()
#hriPesticide.shape

In [21]:
import functools as ft
#extraVars = [hriPesticide, orgProcessors, orgAreaUtil, countryGini, cropProdTotals_Geo_Y, birdBiodiversity, emplyomentRate, income, fertUse, productivityIndex, wasteGeneration]
extraVars = [hriPesticide, orgProcessors, orgAreaUtil, countryGini, birdBiodiversity, emplyomentRate, income, fertUse, productivityIndex, wasteGeneration, pestSales, orgTonne, stdProduction_lin]
rds = ft.reduce(lambda left, right: pd.merge(left,right, how='left', on=['geo', 'TIME_PERIOD']), extraVars)
#rds.shape
#rds.head(20)
#print(extraVars_df.groupby(['TIME_PERIOD', 'geo']).size())
missing_values_table(rds)#Will likely keep NAN values and use as a category when clustered to create scorecard perhaps?

NameError: name 'orgProcessors' is not defined

In [ ]:
rds.to_csv(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Datasets\referenceDataSet.csv")

In [ ]:
cropProdTotals_Geo_Y.to_csv(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Datasets\cropProdTotals_Geo_Y.csv")
#pestSales.to_csv(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Datasets\pestSales.csv")
pestUse.to_csv(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Datasets\pestUse.csv")